### Figure 1 (kernel: R_3.5.3-9.3.0)

In [1]:
source('../code/mfpca_rotation_v2.R')
source('../code/sim_settings_v2.R') # for plotting figures
source('../code/basis_setup_sparse.R')

Loading required package: StanHeaders

Loading required package: ggplot2

rstan (Version 2.21.2, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)



In [2]:
dir = '/panfs/panfs1.ucsd.edu/panscratch/lij014/Oct_2020_mfpca/simulations'
i = 4

In [3]:
mfpca_output = paste0(dir, '/results_est_all/3D_est_all_R_v0_nsims', i, '.Rdata');
dat_sim = paste0(dir, "/sim_data/sim_dat_3B_N_100_R_corr_0_Complete_0.8.Rdata");
idx_sim = i; mean_lim = c(-15,15); FPC_lim = c(-5,5);
fig_name = paste0('../figures/fig1_3D_est_all_R_v0_nsims', i, '.png');

In [4]:
	load(mfpca_output)
	load(dat_sim, dat <- new.env())
	post_rotation_results = post_hoc_rotation(fit=fit, Nchains=Nchains, Nsamples=Nsamples, N=dat_new$num_subjects, 
		                                      P=dat_new$num_blocks, K=dat_new$num_PCs, Q=dat_new$num_basis)

In [5]:
	#dat = dat_sim
	i = idx_sim
	Y_sparse = dat$Y_SPARSE[[i]]
	time_sparse = dat$TIME_SPARSE[[i]]
	results_basis=basis_setup_sparse(dat$TIME_SPARSE[[i]], nknots=dat$sim_param$nknots, orth=TRUE, plotit=FALSE)
	phi_t_cont = results_basis[[5]]
	phi_t = results_basis[[3]]
	time_cont = results_basis[[4]]
	ALPHA_array = post_rotation_results$ALPHA_array
	MU_array = post_rotation_results$MU_array
	THETA_array = post_rotation_results$THETA_array
	COV_array = post_rotation_results$COV_array
	COR_array = post_rotation_results$COR_array	    

	nloop=dim(ALPHA_array)[3]
	first=1
	last=nloop

	MU_mean = MU_array[, first] #mean function across sampling sessions
	ALPHA_mean = ALPHA_array[,,first] # mean factor scores
	THETA_mean = THETA_array[,,first] # mean factor loading
	COV_mean = COV_array[,,first]
	COR_mean = COR_array[,,first]    

	for(iter in 2:nloop){
		MU_mean = MU_mean + MU_array[, iter]
		ALPHA_mean = ALPHA_mean + ALPHA_array[,,iter]
		THETA_mean = THETA_mean + THETA_array[,,iter]
		COV_mean = COV_mean + COV_array[,,iter]
		COR_mean = COR_mean + COR_array[,,iter]        
	}

	MU_mean=cbind(MU_mean/(last-first+1)) 
	ALPHA_mean=cbind(ALPHA_mean/(last-first+1))
	THETA_mean=cbind(THETA_mean/(last-first+1)) 
	COV_mean=cbind(COV_mean/(last-first+1)) 
	COR_mean=cbind(COR_mean/(last-first+1))     

	MU_true = dat$sim_param$params[[7]]
	Mu_true_functions=t(bdiag(cbind(phi_t_cont[[1]]),cbind(phi_t_cont[[2]]), cbind(phi_t_cont[[3]])))%*%MU_true
	Mu_functions=t(bdiag(cbind(phi_t_cont[[1]]),cbind(phi_t_cont[[2]]), cbind(phi_t_cont[[3]])))%*%MU_mean
	Mu1_true=Mu_true_functions[1:length(time_cont)]
	Mu2_true=Mu_true_functions[(length(time_cont)+1):(2*length(time_cont))]
	Mu3_true=Mu_true_functions[(2*length(time_cont)+1):(3*length(time_cont))]
	Mu1=Mu_functions[1:length(time_cont)]
	Mu2=Mu_functions[(length(time_cont)+1):(2*length(time_cont))]
	Mu3=Mu_functions[(2*length(time_cont)+1):(3*length(time_cont))]

	FPC_true = dat$sim_param$params[[8]]

	K = dat$sim_param$Q
	Q = (dat$sim_param$nknots + 4)
	N = dat$sim_param$N
	FPC1_mean=t(phi_t_cont[[1]])%*%THETA_mean[1:Q[1],1:K[1]]
	FPC2_mean=t(phi_t_cont[[2]])%*%THETA_mean[(Q[1]+1):(Q[1]+Q[2]),(K[1]+1):(K[1]+K[2])]
	FPC3_mean=t(phi_t_cont[[3]])%*%THETA_mean[(Q[1]+Q[2]+1):sum(Q),sum(K)]
	time = dat$sim_param$time


Attaching package: ‘pracma’


The following objects are masked from ‘package:Matrix’:

    expm, lu, tril, triu




In [6]:
# commented lines: colors
# updated lines: greyscale

png(fig_name, units="in", width=7, height=5, res=300)
par(mfrow=c(2,3))
#         plot(time_cont,Mu1_true,type="l",ylim=mean_lim,xlim=c(0,1),lwd=1.5,col=4, ylab='response',
#             font.lab=2, cex.lab=1, font.axis=1, xlab='time')
        plot(time_cont,Mu1_true,type="l",ylim=mean_lim,xlim=c(0,1),lwd=1.5,col=1, ylab='response',
            font.lab=2, cex.lab=1, font.axis=1, xlab='time')
        for(i in 1:N){
            lines(time_sparse[[i]][[1]],Y_sparse[[i]][[1]],type="l",lwd=.1)
        }
        #lines(time_cont,Mu1,type="l",col=2,lwd=1.5,lty=2)
        lines(time_cont,Mu1,type="l",col=1,lwd=1.5,lty=2)
        title(main="Outcome for Variable 1")
        #legend('top', c('True mean', 'Estimated'), col=c(4,2), lty=c(1,2), lwd=c(2,2), bty='n')
        legend('top', c('True mean', 'Estimated'), lty=c(1,2), lwd=c(2,2), bty='n')
        fig_label('A', pos='topleft', cex=1.5)

#         plot(time_cont,Mu2_true,type="l",ylim=mean_lim,xlim=c(0,1),lwd=1.5,col=4, ylab='response',
#             font.lab=2, cex.lab=1, font.axis=1, xlab='time')
        plot(time_cont,Mu2_true,type="l",ylim=mean_lim,xlim=c(0,1),lwd=1.5,col=1, ylab='response',
            font.lab=2, cex.lab=1, font.axis=1, xlab='time')
        for(i in 1:N){
            lines(time_sparse[[i]][[2]],Y_sparse[[i]][[2]],type="l",lwd=.1)
        }
#         lines(time_cont,Mu2,type="l",col=2,lwd=1.5,lty=2)
        lines(time_cont,Mu2,type="l",col=1,lwd=1.5,lty=2)
        title(main="Outcome for Variable 2")
        fig_label('B', pos='topleft', cex=1.5)

#         plot(time_cont,Mu3_true,type="l",ylim=mean_lim,xlim=c(0,1),lwd=1.5,col=4, ylab='response',
#             font.lab=2, cex.lab=1, font.axis=1, xlab='time')
        plot(time_cont,Mu3_true,type="l",ylim=mean_lim,xlim=c(0,1),lwd=1.5,col=1, ylab='response',
            font.lab=2, cex.lab=1, font.axis=1, xlab='time')
        for(i in 1:N){
            lines(time_sparse[[i]][[3]],Y_sparse[[i]][[3]],type="l",lwd=.1)
        }
        #lines(time_cont,Mu3,type="l",col=2,lwd=1.5,lty=2)
        lines(time_cont,Mu3,type="l",col=1,lwd=1.5,lty=2)
        title(main="Outcome for Variable 3")
        fig_label('C', pos='topleft', cex=1.5)

        plot(time,FPC_true[,1],type="l",lwd=2,ylim=FPC_lim, ylab='FPC curve',
            font.lab=2, cex.lab=1, font.axis=1)
        lines(time,FPC_true[,2],type="l",lwd=1)
#         lines(time_cont,FPC1_mean[,1],type="l",lwd=2,col=2,lty=2)
#         lines(time_cont,FPC1_mean[,2],type="l",lwd=1,col=2,lty=2)
        lines(time_cont,FPC1_mean[,1],type="l",lwd=2,col=1,lty=2)
        lines(time_cont,FPC1_mean[,2],type="l",lwd=1,col=1,lty=2)
        title(main="PFCs for Variable 1")
#         legend('top', c('True PC', 'Estimated'), col=c(1,2), lty=c(1,2), lwd=c(2,2), bty='n')
        legend('top', c('True PC', 'Estimated'), col=c(1,1), lty=c(1,2), lwd=c(2,2), bty='n')
        fig_label('D', pos='topleft', cex=1.5)

        plot(time,FPC_true[,3],type="l",lwd=2,ylim=FPC_lim, ylab='FPC curve',
            font.lab=2, cex.lab=1, font.axis=1)
        lines(time,FPC_true[,4],type="l",lwd=1)
#         lines(time_cont,FPC2_mean[,1],type="l",lwd=2,col=2,lty=2)
#         lines(time_cont,FPC2_mean[,2],type="l",lwd=1,col=2,lty=2)
        lines(time_cont,FPC2_mean[,1],type="l",lwd=2,col=1,lty=2)
        lines(time_cont,FPC2_mean[,2],type="l",lwd=1,col=1,lty=2)
        title(main="PFCs for Variable 2")
        fig_label('E', pos='topleft', cex=1.5)

        plot(time,FPC_true[,5],type="l",lwd=2,ylim=FPC_lim, ylab='FPC curve',
            font.lab=2, cex.lab=1, font.axis=1)
#         lines(time_cont,FPC3_mean[,1],type="l",lwd=2,col=2,lty=2)
        lines(time_cont,FPC3_mean[,1],type="l",lwd=2,col=1,lty=2)
        title(main="PFCs for Variable 3")
        fig_label('F', pos='topleft', cex=1.5)
dev.off()

png 
  2